## Hierarchical Topic Models and the Nested Chinese Restaurant Process

### I. Background

Recently, complex probabilistic models are increasingly prevalent in various of domains. However, there are several challenges that should be dealt with due to their open-ended nature. That is, the data sets often grow over time, as they growing, they bring new entities and new structures to the fore. Take the problem of learning a topic hierarchy from data for example. Given a collection of __*documents*__, each of which contains a set of __*words*__ and the goal is to discover common usage patterns or __*topics*__ in the documents, and to organize these topics into a hierarchy.

This paper proposes a new method that specified a generative probabilistic model for hierarchical structures and adopt Bayesian perspective to learn such structures from data. The hierarchies in this case could be considered as random variables and specified procedurally. In addition, the underlying approach of constructing the probabilistic object is __Chinese restaurant process (CRP)__, a distribution on partitions of integers. In this paper, they extend CRP to a hierarchy of partitions, known as __nested Chinese restaruant process (nCRP)__, and apply it as a representation of prior and posterior distributions for topic hierarchies. To be more specific, each node in the hierarchy is associated with a topic, where a topic is a distribution across words. A document is generated by choosing a path from the root to a leaf, repeatedly sampling topics along that path, and sampling the words from the selected topics. Thus the orga- nization of topics into a hierarchy aims to capture the breadth of usage of topics across the corpus, reflecting underlying syntactic and semantic notions of generality and specificity.

### II. Algorithm Description

#### A. Chinese Restaurant Process

CRP is an analogous to seating customers at tables in a Chinese restaurant. Imagine there is a Chinese restaurant with an infinite number of circular tables, each with infinite capacity. Customer 1 sits at the first table. The next customer either sits at the same table as customer 1, or the next table. The $m$th subsequent customer sits at a table drawn from the following distribution:

$$p(\text{occupied table } i \text{ | previous customers}) =  \frac{m_i}{\gamma+m-1}$$

$$p(\text{next unoccupied table | previous customers}) = \frac{\gamma}{\gamma + m -1}$$

where $m_i$ is the number of previous customers at table $i$, and $\gamma$ is a parameter. After $M$
customers sit down, the seating plan gives a partition of $M$ items. This distribution gives
the same partition structure as draws from a Dirichlet process.

#### B. Nested Chinese Restaurant Process

A nested Chinese restaurant process (nCRP) is an extended version of CRP. Suppose that there are an infinite number of infinite-table Chinese restaurants in a city. A restaurant is determined to be the root restaurant and on each of its infinite tables is a card with the name of another restaurant. On each of the tables in those restaurants are cards that refer to other restaurants, and this structure repeats infinitely. Each restaurant is referred to exactly once.  As a result, the whole process could be imagined as an infinitely-branched tree.

Now, consider a tourist arrives in the city for a culinary vacation. On the first first day, he select a root Chinese restaurant and selects a table from the equation above. On the second day, he enters to the restaurant refered by previous restaurant , again from the above equation. This process was repeated for $L$ days, and at the end, the tourist has sat at L restaurants which constitute a path from the root to a restaurant at the $L$th level in the infinite tree. After M tourists take L-day vacations, the collection of paths describe a particular L-level subtree of the infinite tree.

#### C. Hierarchical Topic Model (hLDA)

The hierarchical latent Dirichlet allocation model (hLDA) together with nested Chinese restaruant process (nCRP) illustrate the pattern of words from the collection of documents. There are 3 procedures in hLDA: (1) Draw a path from root-node to a leaf; (2) Select a specific path, draw a vector of topic along the path; (3) Draw the words from the topic. In addition, all documents share the topic associated with the root restaurant.

1. Let c1 be the root restaurant.
+ For each level $\ell\in\{2,...,L\}$:
    1. Draw a table from restaurant $c_{\ell-1}$ using CRP. Set $c_{\ell}$ to be the restaurant reffered to by that table.
+ Draw an L-dimensional topic proportion vector $\theta$ from Dir($\alpha$).
+ For each word $n\in\{1,...,N\}$:
    1. Draw $z\in\{1,...,L\}$ from Mult($\theta$).
    + Draw $w_n$ from the topic associated with restaurant $c_z$.

<img src="hLDA.png" style="width:400px">

* Notation:
    * $T$ : L-level infinite-tree - drawn from CRP($\gamma$)
    * $\theta$ : L-dimensional topic propotional distribution - drawn from Dir($\alpha$)
    * $\beta$ : probability of words for each topic - drawn from $\eta$
    * $c_{\ell}$ : L-level paths, given $T$
    * $z$ : actual number of topics for each level - drawn from Mult($\theta$)
    * $w$ : word distribution for each topic at each level
    * $N$ : number of words - $n\in\{1,...,N\}$
    * $M$ : number of documents - $m\in\{1,...,M\}$

### III. Approximate Inference by Gibbs Sampling

Gibbs sampling will sample from the posterior nCRP and corresponding topics in the hLDA model. The sampler are divided into 2 parts -- $z_{m,n}$ and $ c_{m,\ell}$. In addition, variables $\theta$ and $\beta$ are integrated out.

#### A. Notation

* $w_{m,n}$ : the $n$th word in the $m$th documnt
* $c_{m,\ell}$ : the restaurant corresponding to the $\ell$th topic in document $m$
* $z_{m,n}$ : the assignment of the $n$th word in the $m$th document to one of the $L$ available topics

#### B. Topic distribution : $z_{m,n}$

$$p(z_{i}=j\hspace{0.5ex}|\hspace{0.5ex}{\bf z}_{-i},{\bf w})\propto\frac{n_{-i,j}^{(w_{i})}+\beta}{n_{-i,j}^{(\cdot)}+W\beta}\frac{n_{-i,j}^{(d_{i})}+\alpha}{n_{-i,\cdot}^{(d_{i})}+T\alpha}$$

* $z_{i}$ : assignments of words to topics
* $n_{-i,j}^{(w_{i})}$ : number of words assigned to topic $j$ that are the same as $w_i$
* $n_{-i,j}^{(\cdot)}$ : total number of words assigned to topic $j$
* $n_{-i,j}^{(d_{i})}$ : number of words from document $d_i$ assigned to topic $j$
* $n_{-i,\cdot}^{(d_{i})}$ : total number of words in document $d_i$
* $W$ : number of words have been assigned

#### C. Path : ${\bf c}_{m}$

$$p({\bf c}_{m}\hspace{0.5ex}|\hspace{0.5ex}{\bf w}, {\bf c}_{-m}, {\bf z})\propto p({\bf w}_{m}\hspace{0.5ex}|\hspace{0.5ex}{\bf c}, {\bf w}_{-m}, {\bf z})\cdot p({\bf c}_{m}\hspace{0.5ex}|\hspace{0.5ex}{\bf c}_{-m})$$

* $p({\bf c}_{m}\hspace{0.5ex}|\hspace{0.5ex}{\bf w}, {\bf c}_{-m}, {\bf z})$ : posterior of the set of probabilities of possible novel paths
* $p({\bf w}_{m}\hspace{0.5ex}|\hspace{0.5ex}{\bf c}, {\bf w}_{-m}, {\bf z})$ : likelihood of the data given a particular choice of ${\bf c}_{m}$
* $p({\bf c}_{m}\hspace{0.5ex}|\hspace{0.5ex}{\bf c}_{-m})$ : prior on ${\bf c}_{m}$ which implies by the nCRP

$$p({\bf w}_{m}\hspace{0.5ex}|\hspace{0.5ex}{\bf c}, {\bf w}_{-m}, {\bf z})=\prod\limits_{\ell=1}^{L}\left(\frac{\Gamma(n_{c_{m,\ell},-m}^{(\cdot)}+W\eta)}{\prod_{w}\Gamma(n_{c_{m,\ell},-m}^{(w)}+\eta)}\frac{\prod_{w}\Gamma(n_{c_{m,\ell},-m}^{(w)}+n_{c_{m,\ell},m}^{(w)}+\eta)}{\Gamma(n_{c_{m,\ell},-m}^{(\cdot)}+n_{c_{m,\ell},m}^{(\cdot)}+W\eta)}\right)$$

* $p({\bf w}_{m}\hspace{0.5ex}|\hspace{0.5ex}{\bf c}, {\bf w}_{-m}, {\bf z})$ : joint distribution of likelihood
* $n_{c_{m,\ell},-m}^{(w)}$ : number of instances of word $w$ that have been assigned to the topic indexed by $c_{m,\ell}$, not in the document $m$
* $W$ : total vocabulary size